In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame
ep = 1e-7

##--------------------Data Preprocesseing -------------------------------

Train_X = []
Train_Y = []
y=0
for a,b,file in os.walk("horse-or-human/train"):
    for i in file:
        str1 = a+"/"+str(i)
        t = cv2.imread(str1)
        t = cv2.cvtColor(t,cv2.COLOR_BGR2GRAY)
        t = t/255
        t = t.reshape(10000)
        Train_X.append(t)
        Train_Y.append(y-1)
    y+=1

Test_X = []
Test_Y = []
y=0
for a,b,file in os.walk("horse-or-human/validation"):
    for i in file:
        str1 = a+"/"+str(i)
        t = cv2.imread(str1)
        t = cv2.cvtColor(t,cv2.COLOR_BGR2GRAY)
        t = t/255
        t = t.reshape(10000)  #vectorization
        Test_X.append(t)
        Test_Y.append(y-1)
    y+=1
print("loading the train img Complete : %d"%(len(Train_X)))
print("loading the validation img Complete : %d"%(len(Test_X)))


loading the train img Complete : 1027
loading the validation img Complete : 256
